In [1]:
import cirq
import numpy as np
from scipy.optimize import minimize

In [2]:
# Define the quantum circuit (our QNN)
def qnn(params, data_point):
    # Initialize circuit and qubit
    qubit = cirq.GridQubit(0, 0)
    circuit = cirq.Circuit()
    
    # Data encoding gates
    circuit.append(cirq.ry(data_point[0])(qubit))
    circuit.append(cirq.rx(data_point[1])(qubit))
    
    # QNN gates
    circuit.append(cirq.rz(params[0])(qubit))
    circuit.append(cirq.ry(params[1])(qubit))
    circuit.append(cirq.rx(params[2])(qubit))
    circuit.append(cirq.ry(params[3])(qubit))

    # Measurement
    circuit.append(cirq.measure(qubit, key="m"))

    # Simulate the circuit
    simulator = cirq.Simulator()
    result = simulator.run(circuit, repetitions=100)
    measurements = result.measurements['m']
    
    # Taking the expectation value of the Z measurements
    expectation_value = np.mean([1 if x[0] else -1 for x in measurements])
    
    return expectation_value

In [3]:
# Classification based on QNN's output
def classify(data_point, weights):
    raw_output = qnn(weights, data_point)
    return 1 if raw_output > 0 else 0

# Define the cost function
def cost(weights):
    predictions = [classify(point, weights) for point in data]
    loss = sum([(pred - true_label)**2 for pred, true_label in zip(predictions, labels)])
    return loss / len(data)

In [4]:
# Sample quantum-compatible data
data = [[0.5, 0.3], [0.2, 0.4], [0.1, 0.9], [0.7, 0.4]]
labels = [0, 1, 1, 0]

# Randomly initialize weights
initial_weights = np.random.rand(4)

# Training using scipy's minimize function
result = minimize(cost, initial_weights, method="COBYLA")
print("Optimized weights:", result.x)

Optimized weights: [1.78613485 0.19844038 0.09272786 0.66226183]
